In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install category_encoders
# !pip install dacon_submit_api-0.0.4-py3-none-any.whl
# !pip install mljar-supervised

In [3]:
## import 
import pandas as pd
import numpy as np
from supervised.automl import AutoML

# vis
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic' # 한글 패치. 코랩에선 주석하셈
import seaborn as sns

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier, VotingRegressor 
from sklearn.ensemble import StackingClassifier, StackingRegressor
from sklearn.base import ClassifierMixin
# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss,mean_squared_error

# CatBoost
from catboost import CatBoostRegressor

# Utility
import os
import time
import datetime # ⚠️2019년 12월30일과 31일의 week of year가 1인 오류가 있음
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
import holidays

# 시드값 고정
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(2023)

# 데이터프레임 컬럼 전체 보기
pd.set_option('display.max_columns', None)

In [4]:
# 데이터 로드

path = '../data/daegu/automl/'

train_org = pd.read_csv(path+'train.csv')
test_org = pd.read_csv(path+'test.csv')

In [ ]:
# 이희원님 코드 그대로 사용
for df in [train_org, test_org]:
    # 날짜 데이터
    time_pattern = r'(\d{4})-(\d{1,2})-(\d{1,2}) (\d{1,2})'

    df[['연', '월', '일', '시간']] = df['사고일시'].str.extract(time_pattern)
    df[['연', '월', '일', '시간']] = df[['연', '월', '일', '시간']].apply(pd.to_numeric) # 추출된 문자열을 수치화해줍니다
    df['사고일시'] = pd.to_datetime(df['사고일시'])
    # df= df.drop(columns=['사고일시']) # 일단 냄겨두자

    # 도시/구/동 분리
    location_pattern = r'(\S+) (\S+) (\S+)'

    df[['도시', '구', '동']] = df['시군구'].str.extract(location_pattern)
    # df= df.drop(columns=['시군구']) # 얘도 일단 냄겨놔

    # 도로형태도 분류 코드
    #road_pattern = r'(.+) - (.+)'

    #df[['도로형태1', '도로형태2']] = df['도로형태'].str.extract(road_pattern)
    #df= df.drop(columns=['도로형태']) # 얘는 드롭함

    # 시간 >> 코사인 변환
#     df['cos_hour'] = np.cos(2 * np.pi * df['시간'] / 24)
    # 📢송규헌) 사인-코사인 변환
    df['cos_hour'] = np.sign(np.pi*2*df['시간']/24) + np.cos(np.pi*2*df['시간']/24) + np.sign(np.pi*4*df['시간']/24) + np.cos(np.pi*4*df['시간']/24)

    # 월 >> 코사인 변환
#     df['cos_month'] = -np.cos(np.pi*2*df['월']/12) + np.cos(np.pi*2*df['월']/12) - np.cos(np.pi*4*df['월']/12) + np.sign(np.pi*5*df['월']/12) + np.cos(np.pi*1*df['월']/12) + 2

    
    # 공휴일 (주말과 공휴일을 묶음)
    # 공휴일 = 1, 비공휴일 = 0
    holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']
    df['dow'] = df['사고일시'].dt.dayofweek #임시로 만든 컬럼
    df['holiday'] = np.where((df['dow']>=5) | (df['사고일시'].dt.strftime('%Y-%m-%d').isin(holi_weekday)),1,0)
    df.drop('dow',axis=1,inplace=True)
    
    # 계절 >> cos_month 사용할 때는 season은 생략
    def group_season(df):
        df.loc[(df['월'] == 3) | (df['월'] == 4) | (df['월'] == 5), 'season'] = '봄'
        df.loc[(df['월'] == 6) | (df['월'] == 7) | (df['월'] == 8), 'season'] = '여름'
        df.loc[(df['월'] == 9) | (df['월'] == 10) | (df['월'] == 11), 'season'] = '가을'
        df.loc[(df['월'] == 12) | (df['월'] == 1) | (df['월'] == 2), 'season'] = '겨울'
        return df['season']
    df['season'] = group_season(df)

In [ ]:
train_df = train_org
test_df = test_org

In [ ]:
# load data
light_df = pd.read_csv(path+'동별_보안등개수.csv').rename(columns={'설치개수':'보안등개수'})
parking_df = pd.read_csv(path+'주차장급지구분별개수.csv')
speed_df = pd.read_csv(path+'동별_제한속도_2.csv').drop(columns='Unnamed: 0')
cross_walk_df = pd.read_csv(path+'동별_횡단보도개수.csv')
child_area_df = pd.read_csv(path+'동별_어린이보호구역개수_2.csv').rename(columns={'시설종류':'어린이보호구역개수'})


# merge data
train_df = pd.merge(train_df, light_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, parking_df, how='left', on=['구','동'])
train_df = pd.merge(train_df, speed_df, how='left', on='동')
train_df = pd.merge(train_df, cross_walk_df, how='left', on='동')
train_df = pd.merge(train_df, child_area_df, how='left', on=['동'])

test_df = pd.merge(test_df, light_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, parking_df, how='left', on=['구','동'])
test_df = pd.merge(test_df, speed_df, how='left', on='동')
test_df = pd.merge(test_df, cross_walk_df, how='left', on='동')
test_df = pd.merge(test_df, child_area_df, how='left', on=['동'])


In [ ]:
### 피처 추가...

# 시군구별 사고횟수 >> 기본 코드 사용
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])




In [ ]:
###⚠️연도 그냥 타겟 인코딩할래
# # 📢송규헌) 연도값을 연도별 eclo평균값으로 변경
# # 선형추세
# yearly_eclo_avg = train_df.groupby('연')['ECLO'].mean()
# x = np.array(yearly_eclo_avg.index)
# y = yearly_eclo_avg.values
# lr = LinearRegression()
# lr.fit(x.reshape(-1,1),y.reshape(-1,1))
# pred = lr.predict(np.array([2022]).reshape(-1,1))
# yearly_eclo_avg[2022] = pred.reshape(1)[0]

# train_df['연'] =  train_df['연'].map(yearly_eclo_avg)
# test_df['연'] =  test_df['연'].map(yearly_eclo_avg)

In [ ]:
# 점검
display(train_df.head(2))
display(test_df.head(2))

In [ ]:
# 불필요 컬럼 제거
drop_columns = ['ID','도시','시간','시군구','사고일시','사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
       '가해운전자 상해정도','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수'
]
train_df.drop(columns = drop_columns,inplace=True)
test_df.drop(columns=['ID','도시','시간','사고일시','시군구'
                     
                     
                     
                     ],inplace=True)

### 이상치 처리 및 결측값 처리

In [ ]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:11]

In [ ]:
test_df.isnull().sum().sort_values(ascending=False)[:11]

In [ ]:
# # 일단 interpolate
# for df in [train_df,test_df]:
#     df['어린이보호구역개수'] = df['어린이보호구역개수'].interpolate()
#     df['급지구분_1'] = df['급지구분_1'].interpolate()
#     df['급지구분_2'] = df['급지구분_2'].interpolate()
#     df['급지구분_3'] = df['급지구분_3'].interpolate()
#     df['제한속도'] = df['제한속도'].interpolate()
#     df['횡단보도개수'] = df['횡단보도개수'].interpolate()
    
# # 급지구분 interpolate하면 안 채워지는 게 1개 있어서 0으로 대체
# test_df['급지구분_1'] = test_df['급지구분_1'].fillna(0)
# test_df['급지구분_2'] = test_df['급지구분_2'].fillna(0)
# test_df['급지구분_3'] = test_df['급지구분_3'].fillna(0)

In [ ]:
# 결측값을 '구' 별 평균으로 처리
na_columns = ['어린이보호구역개수','급지구분_1','급지구분_1','급지구분_1'
             ,'제한속도','횡단보도개수','보안등개수']


### 어린이보호구역개수 결측값 처리
child_area_mean = train_df.groupby('구')['어린이보호구역개수'].mean()
# 달서구와 동구가 아예 어린이보호구역개수가 nan임.
child_mean_sum = child_area_mean.sum()/len(child_area_mean)
# 전체평균으로 대체 # 동구와 달성군 모두 면적이 넓음
child_area_mean['달서구'] = child_mean_sum
child_area_mean['동구'] = child_mean_sum
def fillna_with_mean_child(row):
    if np.isnan(row['어린이보호구역개수']):
        row['어린이보호구역개수'] = child_area_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_child,axis=1)
test_df = test_df.apply(fillna_with_mean_child,axis=1)


### 급지구분 결측값 처리
parking_mean = train_df.groupby('구')[['급지구분_1','급지구분_2',
                          '급지구분_3']].mean()
cols = ['급지구분_1','급지구분_2','급지구분_3']
for col in cols:
    def fillna_with_mean_parking(row):
        if np.isnan(row[col]):
            row[col] = parking_mean.loc[row['구'],col]
        return row
    train_df = train_df.apply(fillna_with_mean_parking,axis=1)
    test_df = test_df.apply(fillna_with_mean_parking,axis=1)
### 급지구분_1,2,3 컬럼을 주차장지수로 변경 >> 1급지*4 + 2급지*2 + 3급지
# train_df['주차장지수'] = train_df['급지구분_1']*4+train_df['급지구분_2']*2+train_df['급지구분_3']
# test_df['주차장지수'] = test_df['급지구분_1']*4+test_df['급지구분_2']*2+test_df['급지구분_3']    
# train_df.drop(columns=['급지구분_1','급지구분_2','급지구분_3'],inplace=True)
# test_df.drop(columns=['급지구분_1','급지구분_2','급지구분_3'],inplace=True)
    

### 보안등개수 결측값 처리
light_mean = train_df.groupby('구')['보안등개수'].mean()
# 달성군과 서구가 NaN >> 달성군은 면적이 크므로 전체평균값을 활용, 
# 서구는 작으므로 Q1 사용
light_total_mean = light_mean.sum()/len(light_mean)
light_mean['달성군'] = light_total_mean
light_mean['서구'] = light_total_mean * 0.25
def fillna_with_mean_light(row):
    if np.isnan(row['보안등개수']):
        row['보안등개수'] = light_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_light,axis=1)
test_df = test_df.apply(fillna_with_mean_light,axis=1)

 
### 제한속도 결측값 처리
speed_mean = train_df.groupby('구')['제한속도'].mean()
def fillna_with_mean_speed(row):
    if np.isnan(row['제한속도']):
        row['제한속도'] = speed_mean[row['구']]
    return row
train_df = train_df.apply(fillna_with_mean_speed,axis=1)
test_df = test_df.apply(fillna_with_mean_speed,axis=1)


### 횡단보도개수 결측값 처리
# cross_mean = train_df.groupby('구')['횡단보도개수'].mean()
# def fillna_with_mean_cross(row):
#     if np.isnan(row['횡단보도개수']):
#         row['횡단보도개수'] = cross_mean[row['구']]
#     return row
# train_df = train_df.apply(fillna_with_mean_cross,axis=1)
# test_df = test_df.apply(fillna_with_mean_cross,axis=1)
bookgu_cross = train_df[train_df['구']=='북구']['횡단보도개수'].mean()
donggu_cross = train_df[train_df['구']=='동구']['횡단보도개수'].mean()
junggu_cross = train_df[train_df['구']=='중구']['횡단보도개수'].mean()

cross_dict = {'중구': junggu_cross, '동구': donggu_cross, '북구': bookgu_cross}

for i in ['중구', '동구', '북구']:
    train_df.loc[(train_df['구']==i) & (train_df['횡단보도개수'].isnull()), '횡단보도개수'] = cross_dict[i]
    test_df.loc[(test_df['구']==i) & (test_df['횡단보도개수'].isnull()), '횡단보도개수'] = cross_dict[i]


In [ ]:
# 연습용 코드

# ### 제한속도 결측값 처리
# cross_mean = train_df.groupby('구')['횡단보도개수'].mean()
# def fillna_with_mean_speed(row):
#     if np.isnan(row['횡단보도개수']):
#         row['횡단보도개수'] = light_mean[row['구']]
#     return row
# train_df = train_df.apply(fillna_with_mean_speed,axis=1)
# test_df = test_df.apply(fillna_with_mean_speed,axis=1)



In [ ]:
# 결측값 확인 
train_df.isnull().sum().sort_values(ascending=False)[:5]," ---- " , test_df.isnull().sum().sort_values(ascending=False)[:5]

In [ ]:
### 이상치 처리 시각화 코드 무지성 boxplot
i = 0 
outlier_columns = ['보안등개수','횡단보도개수','어린이보호구역개수',
           '사고발생횟수','급지구분_1','급지구분_2','급지구분_3'] # 확인하고픈 수치형 컬럼 넣기
sub_cols = 2
sub_rows = (len(outlier_columns)+1) // 2
width = len(outlier_columns) * 2 ### width, height 곱하는 숫자 조정해서 플롯 사이즈 조정
height = sub_rows * 4

plt.figure(figsize=(width, height)) # figure의 width height는 맘대로
for col in outlier_columns:
    i += 1
    plt.subplot(sub_rows,sub_cols,i)
    sns.boxplot(data=train_df,x=col,gap=.1)
#     plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
### 이상치 처리 함수 정의
import numpy as np
def tukey_fences(data): # data는 1차원 배열
    q1,q3 = np.percentile(data,[25,75])
    iqr = q3-q1
    lf = q1 - (iqr*1.5)
    uf = q3 + (iqr*1.5)
    idxs = np.where((data>uf) | (data<lf))[0] # condition이 True일 때, data의 인덱스를 반환
    return idxs, lf, uf

def outlier_to_lf_uf(data, outlier_function=tukey_fences): # data는 Series객체
    idxs, lf, uf = outlier_function(data)
    for i in idxs:
        if data.iloc[i] <= lf:
            data.iloc[i] = lf
        elif data.iloc[i] >= uf:
            data.iloc[i] = uf
            
    return data

In [ ]:
train_df.columns

In [ ]:
### 이상치 처리 수행
for col in outlier_columns:
    train_df[col] = outlier_to_lf_uf(train_df[col], tukey_fences)
    test_df[col] = outlier_to_lf_uf(test_df[col], tukey_fences)

In [ ]:
### 이상치 처리가 완료된 데이터 시각화
i = 0 

sub_cols = 2
sub_rows = (len(outlier_columns)+1) // 2
width = len(outlier_columns) * 2 ### width, height 곱하는 숫자 조정해서 플롯 사이즈 조정
height = sub_rows * 4

plt.figure(figsize=(width, height)) # figure의 width height는 맘대로
for col in outlier_columns:
    i += 1
    plt.subplot(sub_rows,sub_cols,i)
    sns.boxplot(data=train_df,x=col,gap=.1)
#     plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
train_df.columns

### 인코딩 및 스케일링

In [ ]:
# 원핫 : 사고유형, 도로형태, 동

onehot_columns = ['사고유형', '도로형태', '동']

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[onehot_columns]
test_categorical_data = test_df[onehot_columns]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(onehot_columns)

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

### PCA 
pca = PCA(n_components=98)
train_encoded_df_pca = pd.DataFrame(pca.fit_transform(train_encoded_df))
test_encoded_df_pca = pd.DataFrame(pca.transform(test_encoded_df))

new_col_list = list(map(lambda x:'pca_'+str(x),[i for i in range(98)]))
old_col_list = [i for i in range(98)]
rename_dict = dict(zip(old_col_list, new_col_list))
train_encoded_df_pca.rename(columns=rename_dict,inplace=True)
test_encoded_df_pca.rename(columns=rename_dict,inplace=True)

# 기존 열 제거
train_df = train_df.drop(onehot_columns, axis=1)
test_df = test_df.drop(onehot_columns, axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df_pca], axis=1)
test_df = pd.concat([test_df, test_encoded_df_pca], axis=1)


In [ ]:
pca_practice = PCA()
X = train_encoded_df.copy()
pca_practice.fit(X)

# 누적 설명된 분산 비율 계산
cumulative_variance = np.cumsum(pca_practice.explained_variance_ratio_)

# 90% 이상 설명력을 갖는 주성분 개수 계산
n_components = np.argmax(cumulative_variance >= 0.95) + 1

# 설명된 분산 비율 시각화
plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance, marker='o', linestyle='--', color='b')
plt.xlabel('Number of Components')
plt.ylabel('Cumulative Explained Variance')
plt.title('Cumulative Explained Variance by Number of Components')
plt.axvline(x=n_components, color='r', linestyle=':', label=f'{n_components} components (95% explained variance)')
plt.legend()
plt.grid()
plt.show()

print(f"95% 이상의 설명력을 갖기 위해 {n_components}개의 주성분으로 나누어야 합니다.")


In [ ]:
# 타겟 인코딩 : '요일', '기상상태', '노면상태', '연', '구', season
#        
# Target encoding
from category_encoders.target_encoder import TargetEncoder

### ✔️ 타겟 인코딩하기 전에 eclo 로그 변환해주기
train_df['ECLO'] = np.log1p(train_df['ECLO'])

categorical_features = ['요일', '기상상태', '노면상태', '연', '구','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

In [ ]:

train_df

In [ ]:
# 스케일링 전 전처리
# 일 필요 없을 듯?
# train_df.drop(columns=['일'],inplace=True)
# test_df.drop(columns=['일'],inplace=True)

In [ ]:
# 스케일링 전 분포 시각화
i = 0 
columns = ['어린이보호구역개수','급지구분_1','급지구분_2','급지구분_3',
          '제한속도','횡단보도개수','사고발생횟수','보안등개수']
sub_cols = 2
sub_rows = (len(columns)+1) // 2
width = len(columns)
height = sub_rows * 4

plt.figure(figsize=(width, height)) # figure의 width height는 맘대로
for col in columns:
    i += 1
    plt.subplot(sub_rows,sub_cols,i)
    sns.histplot(data=train_df,x=col,kde=True)
plt.tight_layout()

In [ ]:
columns

In [ ]:
## 스케일링
scaling_columns = ['어린이보호구역개수',
#  '급지구분_1',
#  '급지구분_2',
#  '급지구분_3',
 '제한속도',
 '횡단보도개수',
 '사고발생횟수',
 '보안등개수']

log_scaling_columns = ['급지구분_1','급지구분_2','급지구분_3'] # ECLO는 이미 로그변환 완료

for c in log_scaling_columns:
    train_df[c] = np.log1p(train_df[c])
    test_df[c] = np.log1p(test_df[c])

# scaler = MinMaxScaler()
scaler = StandardScaler()
for c in scaling_columns:
    train_df[c] = scaler.fit_transform(np.reshape(train_df[c],(-1,1)))
    test_df[c] = scaler.transform(np.reshape(test_df[c],(-1,1)))

In [ ]:
# 스케일링 후 분포 시각화
i = 0 
sub_cols = 2
sub_rows = (len(columns)+1) // 2
width = len(columns)
height = sub_rows * 4

plt.figure(figsize=(width, height)) # figure의 width height는 맘대로
for col in columns:
    i += 1
    plt.subplot(sub_rows,sub_cols,i)
    sns.histplot(data=train_df,x=col,kde=True)
plt.tight_layout()

### 기타 전처리

In [ ]:
train_df

In [ ]:
# train_df.to_csv('./train_df.csv',index=False)
# test_df.to_csv('./test_df.csv',index=False)

### 데이터셋 분할

In [ ]:
train_df.head(1)

In [ ]:
# 컬럼명 영어로 바꾸기
kr2eng = {'요일':'dayofweek', '기상상태':'weather', '도로형태':'road_type',
          '노면상태':'road_state', '사고유형':'accident_type', '연':'year',
          '동':'dong', '보안등개수':'light_count', '제한속도':'speed_limit',
          '횡단보도개수':'cross_count', '어린이보호구역개수':'child_count',
          '사고발생횟수':'accident_count', '주차장지수':'parking_index',
          '구_남구':'gu_namgu', '구_달서구':'gu_dalseogu', 
          '구_달성군':'gu_dalsunggun', '구_동구':'gu_donggu', 
          '구_북구':'gu_bookgu', '구_서구':'gu_seogu',
       '구_수성구':'gu_suseonggu', '구_중구':'gu_junggu',
         '월':'month','일':'day','구':'gu',
         '급지구분':'park_1','급지구분_2':'park_2','급지구분_3':'park_3'}

train_df.rename(columns=kr2eng,inplace=True)
test_df.rename(columns=kr2eng,inplace=True)

In [ ]:
train_x = train_df[test_df.columns] # test 셋에만 있는 컬럼만 추출
train_y = train_df['ECLO']
test_x = test_df

In [ ]:
# x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### AUTOML

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
from supervised.automl import AutoML
automl = AutoML(
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=40000, eval_metric="rmse", ml_task = "regression",
               results_path = './AutoML_005_withScale_Compete/',
                explain_level=2,
                mode='Compete',
                random_state=42,
                stack_models=False
               )

In [ ]:
automl.fit(train_x, train_y)

In [ ]:
pred = automl.predict(test_x)

## 제출

In [ ]:
submission = pd.read_csv(path+'sample_submission.csv')

In [ ]:
submission['ECLO'] = np.expm1(pred)

In [ ]:
submission.loc[ submission['ECLO'] < 0.0, 'ECLO'] = 0.0

In [ ]:
min(submission['ECLO'])

In [ ]:
max(submission['ECLO'])

In [ ]:
submission

In [ ]:
submission.to_csv('submit.csv', index=False)

submit_path = "../data/submits/automl_1209_005_withScale_Compete_noStacking.csv"
submission.to_csv(submit_path, index=False)

In [ ]:
# 데이콘 api 제출

from dacon_submit_api import dacon_submit_api

result = dacon_submit_api.post_submission_file(
  submit_path,  # 제출 파일의 저장경로
  '99e503e9d7311db9f4167d982d5823d7ef9427cc628aa63b96cf79d2920ba916', # API 토큰
  '236193', # 대구공모전 ID
  'song999', # 맘대루
  '12.09 - 005_withScale. automl_compete_noStacking' )  # 제출 메모